In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

from ta.volume import MFIIndicator

from copy import copy

In [5]:
# Read recipe inputs
yfinance_ndx_hist_1985_daily = dataiku.Dataset("yfinance_ndx_hist_1985_daily")
df_source = yfinance_ndx_hist_1985_daily.get_dataframe()

In [3]:
''' Selon MFI Performance v5, toutes les valeurs suivantes surperforment le Buy & Hold.
Elles ont toutefois des périodes allant du simple au triple,
et un nombre de trades différent'''
mfi_list = [10,14,17,37]

In [6]:
df_mfi = df_source.copy()

In [7]:
for w in mfi_list :
    s_mfi = MFIIndicator(
        high = df_source.High, 
        low = df_source.Low, 
        close = df_source.Close, 
        volume = df_source.Volume, 
        window = w
    ).money_flow_index()
    
    # Ajout de la série en tant que colonne + arrondi
    # to_list() semble nécessaire dans cette version de Python ?!
    df_mfi['MFI_'+str(w)] = round(s_mfi, 2).to_list()
    
    # Pointeur sur la période maximale
    w_max = w

In [8]:
# Démarrage lorsque tous les MFI sont renseignés
df_mfi = df_mfi.loc[(w_max-1):]
df_mfi.reset_index(drop=True, inplace=True)

In [9]:
# Sélection des 4 dernières colonnes qui sont celles des calculs MFI
# cf MFI list length, en négatif
nb_cols = - len(mfi_list)
colonnes_mfi = df_mfi.columns[nb_cols:].to_list()

In [11]:
# Périodes qui vont être utilisées pour les calculs de MA
periodes = [3,5]

In [12]:
for colonne in colonnes_mfi : 
    '''Calcul des moyennes mobiles, MA3 et MA5,
    pour chacune des colonnes MFI'''
    
    # SMA 3
    col_name_short = str(colonne) + "_SMA" + str(periodes[0])
    df_mfi[col_name_short] = round( df_mfi[colonne].rolling(periodes[0]).mean() ,2)
    
    # SMA 5
    col_name_mid = str(colonne) + "_SMA" + str(periodes[1])
    df_mfi[col_name_mid] = round( df_mfi[colonne].rolling(periodes[1]).mean() ,2)

In [14]:
df_mfi.dropna(how='any', inplace=True)
df_mfi.reset_index(drop=True, inplace=True)

In [17]:
# Write recipe outputs
yf1985_MFI = dataiku.Dataset("yf1985_MFI")
yf1985_MFI.write_with_schema(df_mfi)

9213 rows successfully written (TTwZHy8Hwg)
